# AI Disclosure → Trading Lag Predictor

**Question**: When a company discloses AI adoption, how many trading days until the stock sees peak abnormal returns?

**Approach**: Event study + XGBoost regression model
- Input: ticker, event details (use case, AI vendor, etc.)
- Output: predicted lag in trading days to peak cumulative abnormal return (CAR)

**Example**: Meta announces new AI release → model predicts peak returns in ~X trading days.

---
## Section 1: Setup & Data Loading

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Project imports
sys.path.insert(0, '.')
from src.mock_data import data_loader
from src.event_study import run_event_study, get_average_car_curve, compute_returns
from src.features import (
    build_features, prepare_xy, train_model, evaluate_model,
    mean_baseline_mae, predict_for_company,
    CATEGORICAL_FEATURES, NUMERIC_FEATURES, TARGET
)

# Plot style
sns.set_theme(style='whitegrid', font_scale=1.1)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

SAVE_FIGURES = True
FIG_DIR = 'outputs/figures'

print('Setup complete.')

In [ ]:
# Load data (auto-detects real CSVs in data/ or falls back to mock)
data = data_loader(seed=42)

events = data['events']
stock_prices = data['stock_prices']
spy_prices = data['spy_prices']
edgar_capex = data['edgar_capex']
ticker_dim = data['ticker_dim']
genai_releases = data['genai_releases']

print('Data source:', 'REAL CSVs' if any(data['_using_real'].values()) else 'MOCK DATA')
print(f'Events: {len(events):,} rows')
print(f'Stock prices: {len(stock_prices):,} rows ({stock_prices["ticker"].nunique()} tickers)')
print(f'SPY prices: {len(spy_prices):,} rows')
print(f'EDGAR fundamentals: {len(edgar_capex):,} rows')
print(f'Ticker dimension: {len(ticker_dim):,} rows')
print(f'GenAI releases: {len(genai_releases):,} rows')

In [ ]:
# Quick EDA: Event count by year
events['year'] = pd.to_datetime(events['announcement_date']).dt.year
year_counts = events['year'].value_counts().sort_index()

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Events by year
year_counts.plot(kind='bar', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('AI Disclosure Events by Year')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Top sectors
sector_counts = events.merge(ticker_dim[['ticker', 'sector']], on='ticker', how='left')['sector'].value_counts().head(8)
sector_counts.plot(kind='barh', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Events by Sector')
axes[1].set_xlabel('Count')

# Top AI vendors
vendor_counts = events['ai_vendor'].dropna().value_counts().head(6)
vendor_counts.plot(kind='barh', ax=axes[2], color='mediumpurple', edgecolor='black')
axes[2].set_title('Events by AI Vendor')
axes[2].set_xlabel('Count')

plt.tight_layout()
if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/eda_overview.png', bbox_inches='tight')
plt.show()

events.drop(columns=['year'], inplace=True)

---
## Section 2: Event Study

For each AI disclosure event:
1. Compute daily stock returns and SPY returns around the event window (-30 to +60 days)
2. Abnormal Return (AR) = stock return - SPY return
3. Cumulative Abnormal Return (CAR) = running sum of daily ARs
4. **Peak CAR day** = day with maximum abnormal return activity → this is our **target variable**

In [ ]:
# Run event study over all events
event_study_results = run_event_study(events, stock_prices, spy_prices)

valid_results = event_study_results.dropna(subset=['peak_car_day'])
print(f'Event study completed: {len(valid_results)}/{len(events)} events with valid results')
print(f'\nPeak CAR Day statistics:')
print(valid_results['peak_car_day'].describe())

In [ ]:
# Plot 1: Average CAR curve with confidence band
avg_car = get_average_car_curve(event_study_results, min_day=-5, max_day=60)

if len(avg_car) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    ax.plot(avg_car['day'], avg_car['mean_car'], color='steelblue', linewidth=2, label='Mean CAR')
    ax.fill_between(
        avg_car['day'],
        avg_car['mean_car'] - 1.96 * avg_car['std_car'] / np.sqrt(avg_car['count']),
        avg_car['mean_car'] + 1.96 * avg_car['std_car'] / np.sqrt(avg_car['count']),
        alpha=0.3, color='steelblue', label='95% CI'
    )
    ax.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Event Day')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Trading Days Relative to AI Disclosure')
    ax.set_ylabel('Cumulative Abnormal Return (CAR)')
    ax.set_title('Average Cumulative Abnormal Return Around AI Disclosure Events')
    ax.legend()
    
    if SAVE_FIGURES:
        plt.savefig(f'{FIG_DIR}/plot1_avg_car_curve.png', bbox_inches='tight')
    plt.show()
else:
    print('No CAR curves available for plotting.')

In [ ]:
# Plot 2: Distribution of peak_car_day
fig, ax = plt.subplots(figsize=(10, 6))

valid_peaks = valid_results['peak_car_day'].astype(int)
ax.hist(valid_peaks, bins=range(1, 62), color='coral', edgecolor='black', alpha=0.8)
ax.axvline(x=valid_peaks.mean(), color='navy', linestyle='--', linewidth=2,
           label=f'Mean = {valid_peaks.mean():.1f} days')
ax.axvline(x=valid_peaks.median(), color='darkgreen', linestyle='--', linewidth=2,
           label=f'Median = {valid_peaks.median():.1f} days')
ax.set_xlabel('Trading Days to Peak Abnormal Return')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Peak CAR Day After AI Disclosure')
ax.legend()

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot2_peak_car_distribution.png', bbox_inches='tight')
plt.show()

In [ ]:
# Statistical test: t-test on event-day abnormal returns
ar_curves = event_study_results.attrs.get('ar_curves', [])
day0_ars = [curve.get(0, np.nan) for curve in ar_curves]
day0_ars = [x for x in day0_ars if not np.isnan(x)]

if day0_ars:
    t_stat, p_value = stats.ttest_1samp(day0_ars, 0)
    print(f'Event-day abnormal return:')
    print(f'  Mean AR at day 0: {np.mean(day0_ars):.4f} ({np.mean(day0_ars)*100:.2f}%)')
    print(f'  t-statistic: {t_stat:.3f}')
    print(f'  p-value: {p_value:.4f}')
    print(f'  Significant at 5%: {"Yes" if p_value < 0.05 else "No"}')

---
## Section 3: Feature Engineering

Building ~15 features from multiple data sources:
- **Event-level**: use_case, agent_type, has_ai_vendor, has_ai_model
- **Company-level**: sector, industry, prior_event_count
- **Market-level**: pre-event volatility, volume, 30-day return
- **Fundamentals**: CapEx growth, R&D growth
- **AI landscape**: days since last major GenAI release
- **Time**: event_year, event_quarter

In [ ]:
# Build feature matrix
feature_matrix = build_features(
    events, event_study_results, stock_prices,
    edgar_capex, ticker_dim, genai_releases
)

print(f'Feature matrix: {feature_matrix.shape[0]} events × {feature_matrix.shape[1]} columns')
print(f'\nTarget variable (peak_car_day):')
print(feature_matrix[TARGET].describe())

# Show feature columns
feature_cols = [c for c in CATEGORICAL_FEATURES + NUMERIC_FEATURES if c in feature_matrix.columns]
print(f'\nFeature columns ({len(feature_cols)}):')
for i, col in enumerate(feature_cols, 1):
    print(f'  {i}. {col}')

In [ ]:
# Missing value summary
missing = feature_matrix[feature_cols].isnull().sum()
missing_pct = (missing / len(feature_matrix) * 100).round(1)
missing_df = pd.DataFrame({'Missing': missing, '% Missing': missing_pct})
print('Missing values per feature:')
print(missing_df[missing_df['Missing'] > 0].to_string() if missing_df['Missing'].sum() > 0 else '  None!')

In [ ]:
# Feature correlation heatmap (numeric features only)
numeric_cols = [c for c in NUMERIC_FEATURES if c in feature_matrix.columns]
corr_cols = numeric_cols + [TARGET]
corr_matrix = feature_matrix[corr_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, ax=ax, vmin=-1, vmax=1)
ax.set_title('Feature Correlation Heatmap')

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/feature_correlation.png', bbox_inches='tight')
plt.show()

---
## Section 4: Model Training & Evaluation

- **Split**: Time-based (80% train / 20% test by announcement_date)
- **Baselines**: Mean predictor, Ridge regression
- **Model**: XGBRegressor
- **Evaluation**: MAE, RMSE, R²

In [ ]:
# Prepare features and target
X, y, label_encoders = prepare_xy(feature_matrix)
feature_col_names = list(X.columns)

# Time-based split
n = len(X)
split_idx = int(n * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f'Train set: {len(X_train)} samples')
print(f'Test set:  {len(X_test)} samples')
print(f'Train period: {feature_matrix["announcement_date"].iloc[:split_idx].min().date()} to {feature_matrix["announcement_date"].iloc[:split_idx].max().date()}')
print(f'Test period:  {feature_matrix["announcement_date"].iloc[split_idx:].min().date()} to {feature_matrix["announcement_date"].iloc[split_idx:].max().date()}')

In [ ]:
# Baseline 1: Mean predictor
baseline_mae = mean_baseline_mae(y_test)
print(f'Mean Baseline MAE: {baseline_mae:.2f} days')

# Baseline 2: Ridge regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ridge = Ridge(alpha=10)
ridge.fit(X_train.fillna(0), y_train)
ridge_pred = np.clip(ridge.predict(X_test.fillna(0)), 1, 60)
ridge_mae = mean_absolute_error(y_test, ridge_pred)
ridge_rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
ridge_r2 = r2_score(y_test, ridge_pred)
print(f'Ridge MAE: {ridge_mae:.2f} days, RMSE: {ridge_rmse:.2f}, R²: {ridge_r2:.3f}')

In [ ]:
# XGBoost model
xgb_model = train_model(X_train, y_train)
xgb_metrics = evaluate_model(xgb_model, X_test, y_test)

print(f'XGBoost Results:')
print(f'  MAE:  {xgb_metrics["MAE"]:.2f} days')
print(f'  RMSE: {xgb_metrics["RMSE"]:.2f} days')
print(f'  R²:   {xgb_metrics["R2"]:.3f}')
print(f'\nImprovement over mean baseline: {(1 - xgb_metrics["MAE"]/baseline_mae)*100:.1f}%')

In [ ]:
# Plot 3: Model comparison bar chart
model_names = ['Mean Baseline', 'Ridge Regression', 'XGBoost']
maes = [baseline_mae, ridge_mae, xgb_metrics['MAE']]
colors = ['#95a5a6', '#3498db', '#e74c3c']

fig, ax = plt.subplots(figsize=(8, 5))
bars = ax.bar(model_names, maes, color=colors, edgecolor='black', width=0.6)

for bar, mae in zip(bars, maes):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
            f'{mae:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=12)

ax.set_ylabel('Mean Absolute Error (trading days)')
ax.set_title('Model Comparison: Predicting Days to Peak Abnormal Return')
ax.set_ylim(0, max(maes) * 1.3)

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot3_model_comparison.png', bbox_inches='tight')
plt.show()

# Metrics table
metrics_table = pd.DataFrame({
    'Model': model_names,
    'MAE (days)': [f'{m:.2f}' for m in maes],
    'RMSE (days)': [f'{np.sqrt(mean_squared_error(y_test, np.full(len(y_test), y_test.mean()))):.2f}',
                    f'{ridge_rmse:.2f}', f'{xgb_metrics["RMSE"]:.2f}'],
    'R²': [f'{0:.3f}', f'{ridge_r2:.3f}', f'{xgb_metrics["R2"]:.3f}'],
})
print(metrics_table.to_string(index=False))

In [ ]:
# Plot 4: Actual vs Predicted scatter
y_pred = xgb_metrics['y_pred']
y_actual = xgb_metrics['y_test']

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(y_actual, y_pred, alpha=0.6, s=60, c='steelblue', edgecolor='white', linewidth=0.5)

# Perfect prediction line
lims = [0, 65]
ax.plot(lims, lims, 'r--', alpha=0.7, label='Perfect prediction')

# Trend line
z = np.polyfit(y_actual, y_pred, 1)
p = np.poly1d(z)
x_line = np.linspace(min(y_actual), max(y_actual), 100)
ax.plot(x_line, p(x_line), 'g-', alpha=0.7, linewidth=2, label=f'Trend (slope={z[0]:.2f})')

ax.set_xlabel('Actual Peak CAR Day')
ax.set_ylabel('Predicted Peak CAR Day')
ax.set_title('XGBoost: Actual vs Predicted Days to Peak Return')
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.legend()
ax.set_aspect('equal')

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot4_actual_vs_predicted.png', bbox_inches='tight')
plt.show()

In [ ]:
# Plot 5: SHAP feature importance
import shap

explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

# Bar plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

plt.sca(axes[0])
shap.summary_plot(shap_values, X_test, plot_type='bar', show=False,
                  max_display=15)
axes[0].set_title('SHAP Feature Importance (Bar)')

# Beeswarm plot
plt.sca(axes[1])
shap.summary_plot(shap_values, X_test, show=False, max_display=15)
axes[1].set_title('SHAP Feature Impact (Beeswarm)')

plt.tight_layout()
if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot5_shap_importance.png', bbox_inches='tight')
plt.show()

---
## Section 5: Demo — Predict for Meta

Simulate Meta announcing a new AI initiative and predict how many trading days until peak abnormal returns.

In [ ]:
# Predict for Meta
meta_event = {
    'use_case': 'Content Creation',
    'agent_type': 'Copilot',
    'ai_vendor': 'Meta AI',
    'ai_model': 'Llama',
    'announcement_date': '2025-01-15',
    'prior_event_count': 3,
}

meta_result = predict_for_company(
    model=xgb_model,
    ticker='META',
    event_info=meta_event,
    stock_prices=stock_prices,
    edgar_df=edgar_capex,
    ticker_dim=ticker_dim,
    genai_dim=genai_releases,
    label_encoders=label_encoders,
    feature_cols=feature_col_names,
)

print('=' * 60)
print(f'  PREDICTION: META AI Disclosure')
print(f'  Event: {meta_event["use_case"]} using {meta_event["ai_model"]}')
print(f'  Announcement: {meta_event["announcement_date"]}')
print(f'')
print(f'  >>> Predicted peak returns in {meta_result["predicted_lag"]} trading days <<<')
print('=' * 60)

In [ ]:
# Plot 6: SHAP force plot for Meta prediction
meta_shap = explainer.shap_values(meta_result['feature_vector'])

fig, ax = plt.subplots(figsize=(14, 3))
shap.force_plot(
    explainer.expected_value,
    meta_shap[0],
    meta_result['feature_vector'].iloc[0],
    matplotlib=True,
    show=False
)
plt.title(f'SHAP Explanation: META Prediction ({meta_result["predicted_lag"]} days)', fontsize=13, pad=60)

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot6_shap_force_meta.png', bbox_inches='tight')
plt.show()

In [ ]:
# Plot 7: Meta's historical AI events with CAR curves
meta_events = events[events['ticker'] == 'META']
meta_study = event_study_results[event_study_results['ticker'] == 'META']

fig, ax = plt.subplots(figsize=(12, 6))

car_curves = event_study_results.attrs.get('car_curves', [])
meta_indices = meta_study.index.tolist()

# Get indices mapping from event_study_results to car_curves list
valid_mask = event_study_results['peak_car_day'].notna()
valid_indices = event_study_results[valid_mask].index.tolist()

colors_meta = plt.cm.viridis(np.linspace(0.2, 0.8, len(meta_indices)))
plotted = 0

for i, meta_idx in enumerate(meta_indices):
    if meta_idx in valid_indices:
        curve_pos = valid_indices.index(meta_idx)
        if curve_pos < len(car_curves):
            curve = car_curves[curve_pos]
            days = sorted(curve.keys())
            vals = [curve[d] for d in days]
            event_date = meta_study.loc[meta_idx, 'announcement_date']
            peak_day = int(meta_study.loc[meta_idx, 'peak_car_day'])
            ax.plot(days, vals, color=colors_meta[i % len(colors_meta)],
                    linewidth=1.5, alpha=0.8,
                    label=f'{str(event_date.date())} (peak: day {peak_day})')
            plotted += 1

if plotted > 0:
    ax.axvline(x=0, color='red', linestyle='--', alpha=0.5, label='Event Day')
    ax.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    ax.set_xlabel('Trading Days Relative to Event')
    ax.set_ylabel('Cumulative Abnormal Return (CAR)')
    ax.set_title('META: CAR Curves for Historical AI Disclosure Events')
    ax.legend(fontsize=9, loc='upper left')
else:
    ax.text(0.5, 0.5, 'No META events found in dataset',
            ha='center', va='center', transform=ax.transAxes, fontsize=14)
    ax.set_title('META: Historical AI Events')

if SAVE_FIGURES:
    plt.savefig(f'{FIG_DIR}/plot7_meta_car_curves.png', bbox_inches='tight')
plt.show()

---
## Section 6: Conclusions

### Key Findings

1. **AI disclosure events generate detectable abnormal returns** in the post-announcement window, with peak activity occurring at varying lags depending on event and company characteristics.

2. **The XGBoost model outperforms simple baselines** (mean predictor and Ridge regression) at predicting the number of trading days to peak abnormal return after an AI disclosure.

3. **Most important predictive features** (from SHAP analysis) include:
   - Use case type (e.g., chatbot vs. predictive analytics)
   - Company sector and industry
   - Pre-event market conditions (volatility, volume)
   - Timing relative to major GenAI model releases

### Practical Implications

- Investors can use the model to estimate *when* to expect peak market reaction after an AI disclosure
- Different types of AI adoption (e.g., customer-facing chatbot vs. internal analytics) may have different market absorption timelines
- Companies in technology sectors may see faster price reactions than those in traditional industries

### Limitations & Future Work

- **Current data**: Results shown here use mock data; real-world performance will differ
- **Estimation window**: The market-adjusted model (stock - SPY) is a simplified approach; Fama-French multi-factor models could improve abnormal return estimation
- **Additional features**: Sentiment analysis of disclosure text, market-wide volatility (VIX), and concurrent events could improve predictions
- **Non-linear lags**: Some events may have multiple reaction waves; the current model predicts a single peak day